In [2]:
import torch
import clip
from PIL import Image
from utils import *
from tqdm import tqdm
from dataloader_comma import *
import json
from torch.utils.data import DataLoader
import pandas as pd
import os 
import matplotlib.pyplot as plt

In [38]:
p = '/data3/jessica/data/toyota/explanation/'
top3 = []
for elem in os.listdir(p):
    if ".json" not in elem: continue
    with open(p + elem, 'r') as f:
    # Load the data from the file into a Python dictionary:
        data = json.load(f)
    df = pd.DataFrame(list(data.items()), columns=['Text', 'Probability'])
    df["Probability"] = df["Probability"].apply(lambda x: float(x))
    top3.extend(df.sort_values("Probability", ascending=False)[0:5].Text.tolist())

In [40]:
df = pd.DataFrame(top3)
df['count'] = 1

In [41]:
df.groupby(0).sum()

,count
0,
a picture a car in the lane in front which changes the lane,99
a picture of a car close by on the left,3725
a picture of a car close by on the right,2867
a picture of a street with poor visibility in front,11271
a picture of a truck in the lane in front,7
a picture of a truck in the lane in front which changes the lane,1
a picture of a truck on the left,2
a picture of a truck on the right,10
a picture of an obstacle in the lane in front,1


In [34]:
df

,0
0,a picture of a street with poor visibility in ...
1,a picture of cars driving during the day
2,a picture of cars on a street in rain
3,a picture of cars driving during the day
4,a picture of cars waiting at traffic light
...,...
39745,a picture of a car close by on the right
39746,a picture of cars driving in the dark
39747,a picture of a street with poor visibility in ...
39748,a picture of cars driving during the day


In [ ]:
import torch
import clip
from PIL import Image
from utils import *
import pandas as pd
from tqdm import tqdm
from dataloader_comma import *
import json
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

device = "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
multitask = "distance"
dataset_type = 'train'
dataset = CommaDataset(dataset_type=dataset_type, multitask=multitask, return_full=True)
loader = DataLoader(dataset, batch_size=1, num_workers=5)
save_path = '/data3/jessica/data/toyota/explanation_reinstate_cc/'

text = clip.tokenize(scenarios).to(device)
scenarios_tokens = scenarios_tokens.to(device)
with torch.no_grad():
    for j, batch in tqdm(enumerate(loader)):
        image_array, vego, angle, distance, gas, brake, ccenabled = batch
        intervention = ccenabled
        if intervention.sum() == 0: 
            continue
        switch_indices = np.where(np.diff(ccenabled) == 1)[0] + 1

        intervention = np.zeros_like(signal, dtype=bool)
        intervention[switch_indices] = True

        images = image_array.squeeze()[intervention.squeeze().tolist()]
        if images.shape[0] == 0: 
            del image_array, vego, angle, distance, gas, brake
            continue

        
        img = images
        s = img.shape#[batch_size, seq_len, h,w,c]
        logits_per_image, logits_per_text = model(img.to(device), scenarios_tokens.to(device))
        probs = logits_per_image.detach()
        probs = logits_per_image.softmax(dim=-1).cpu().detach()
        df = pd.DataFrame([scenarios[i.item()] for i in probs.argmax(dim=-1)])
        df.to_csv(f'{save_path}/{j}.csv')
        
        for i, img in enumerate(images):
            fig, ax = plt.subplots()
            ax.imshow(img.permute(1,2,0).int())
            fig.suptitle(str(df.iloc[i]))
            name = save_path + f"{i}_{j}" + "_image.png"
            result_dict = {scenarios[k]: str(probs.squeeze()[i][k].item()) for k in range(len(scenarios))}
            with open(name.replace('_image.png', ".json"), 'w') as f:
                json.dump(result_dict, f)
            fig.savefig(name)
        


In [46]:
import numpy as np

# Example boolean array
signal = np.array([0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1])

# Find the indices when the signal switches from 0 to 1
switch_indices = np.where(np.diff(signal) == 1)[0] + 1

# Print the switch indices
print(switch_indices)

[1 3 7 9]
